# Football World Cup 2022

In [79]:
import pandas as pd
import pickle
from scipy.stats import poisson

In [80]:
datamatches = pd.read_csv("/Users/hilmanyusoh/Documents/PWFWCup 2022/clean_fifa_worldcup_matches.csv")
datafixture = pd.read_csv("/Users/hilmanyusoh/Documents/PWFWCup 2022/clean_fifa_worldcup_fixture.csv")

# 1.Calulate team strenght

In [81]:
# split dataframe into datahome and dataway
datahome = datamatches[["HomeTeam","HomeGoals","AwayTeam"]]
dataaway = datamatches[["AwayTeam","HomeGoals","AwayTeam"]]

In [84]:
# rename columns
datahome = datahome.rename(columns={"HomeTeam": "Team", "HomeGoals": "GoalsScored", "AwayGoals": "GoalsConceded"})
dataaway = dataaway.rename(columns={"AwayTeam": "Team", "HomeGoals": "GoalsConceded", "AwayGoals": "GaolsScored"})
                         

In [86]:
datahome

,Team,GoalsScored,AwayTeam
0,France,4,Mexico
1,Uruguay,4,Argentina
2,Uruguay,6,Yugoslavia
3,Argentina,6,United States
4,Paraguay,1,Belgium
...,...,...,...
895,Brazil,2,Costa Rica
896,Serbia,1,Switzerland
897,Serbia,0,Brazil
898,France,1,Peru


In [87]:
dataaway

,Team,GoalsConceded,Team
0,Mexico,4,Mexico
1,Argentina,4,Argentina
2,Yugoslavia,6,Yugoslavia
3,United States,6,United States
4,Belgium,1,Belgium
...,...,...,...
895,Costa Rica,2,Costa Rica
896,Switzerland,1,Switzerland
897,Brazil,0,Brazil
898,Peru,1,Peru
